# Train on W-jets

In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

import tensorflow.keras as keras
import tensorflow.keras.backend as K
# from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization
# from tensorflow.keras.layers import Conv1D
# from tensorflow.keras.layers import Flatten, Reshape, Lambda
# from tensorflow.keras.utils import plot_model
# from tensorflow.keras import Model

import os
import os.path as osp
import sys

import numpy as np
#from scipy import linalg as LA

import matplotlib
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from utils.tf_sinkhorn import ground_distance_tf_nograd, sinkhorn_knopp_tf_scaling_stabilized_class
import utils.VAE_model_tools
from utils.VAE_model_tools import build_and_compile_annealing_vae, betaVAEModel, reset_metrics

import pandas
import matplotlib.pyplot as plt

import h5py
import pickle


1 Physical GPUs, 1 Logical GPUs


In [2]:
def create_dir(dir_path):
    ''' Creates a directory (or nested directories) if they don't exist.
    '''
    if not osp.exists(dir_path):
        os.makedirs(dir_path)

    return dir_path

def ptetaphiE_to_Epxpypz(jets):
    pt = jets[:,:,0]
    eta = jets[:,:,1]
    phi = jets[:,:,2]
    E = jets[:,:,3]
    
    px = pt * np.cos(phi)
    py = pt * np.sin(phi)
    pz = pt * np.sinh(eta)
    
    newjets = np.zeros(jets.shape)
    newjets[:,:,0] = E
    newjets[:,:,1] = px
    newjets[:,:,2] = py
    newjets[:,:,3] = pz
    
    return newjets

def ptetaphiE_to_ptyphim(jets):
    pt = jets[:,:,0]
    eta = jets[:,:,1]
    phi = jets[:,:,2]
    E = jets[:,:,3]
    
    pz = pt * np.sinh(eta)
    y = 0.5*np.nan_to_num(np.log((E+pz)/(E-pz)))
    
    msqr = np.square(E)-np.square(pt)-np.square(pz)
    msqr[np.abs(msqr) < 1e-6] = 0
    m = np.sqrt(msqr)
    
    newjets = np.zeros(jets.shape)
    newjets[:,:,0] = pt
    newjets[:,:,1] = y
    newjets[:,:,2] = phi
    newjets[:,:,3] = m
    
    return newjets
    
def ptyphim_to_ptetaphiE(jets):
    
    pt = jets[:,:,0]
    y = jets[:,:,1]
    phi = jets[:,:,2]
    m = jets[:,:,3]
    
    eta = np.nan_to_num(np.arcsinh(np.sinh(y)*np.sqrt(1+np.square(m/pt))))
    pz = pt * np.sinh(eta)
    E = np.sqrt(np.square(pz)+np.square(pt)+np.square(m))
    
    newjets = np.zeros(jets.shape)
    newjets[:,:,0] = pt
    newjets[:,:,1] = eta
    newjets[:,:,2] = phi
    newjets[:,:,3] = E
    
    return newjets
    
def center_jets_ptetaphiE(jets):
    cartesian_jets = ptetaphiE_to_Epxpypz(jets)
    sumjet_cartesian = np.sum(cartesian_jets,axis=1)
    
    sumjet_phi = np.arctan2(sumjet_cartesian[:,2],sumjet_cartesian[:,1])
    sumjet_y = 0.5*np.log((sumjet_cartesian[:,0] + sumjet_cartesian[:,-1])/(sumjet_cartesian[:,0] - sumjet_cartesian[:,-1]))
    
    ptyphim_jets = ptetaphiE_to_ptyphim(jets)
    #print(ptyphim_jets[:3,:,:])
    
    transformed_jets = np.copy(ptyphim_jets)
    transformed_jets[:,:,1] = ptyphim_jets[:,:,1] - sumjet_y[:,None]
    transformed_jets[:,:,2] = ptyphim_jets[:,:,2] - sumjet_phi[:,None]
    transformed_jets[:,:,2] = transformed_jets[:,:,2] + np.pi
    transformed_jets[:,:,2] = np.mod(transformed_jets[:,:,2],2*np.pi)
    transformed_jets[:,:,2] = transformed_jets[:,:,2] - np.pi

    transformed_jets[transformed_jets[:,:,0] == 0] = 0
    
    newjets = ptyphim_to_ptetaphiE(transformed_jets)
    return newjets
    

## Load and preprocess train/val data

In [4]:
# path to file
fn =  '/home/jcollins/projects/EMD_VAE/in_data/monoW-data-3.h5'
# fn =  '/media/jcollins/MAGIC!/monoW-data-3.h5'

# Option 1: Load everything into memory
df = pandas.read_hdf(fn,stop=1000000)
print(df.shape)
print("Memory in GB:",sum(df.memory_usage(deep=True)) / (1024**3)+sum(df.memory_usage(deep=True)) / (1024**3))

(602055, 200)
Memory in GB: 1.803235039114952


In [5]:
# Data file contains, for each event, 50 particles (with zero padding), each particle with pT, eta, phi, E.
data = df.values.reshape((-1,50,4))

# Normalize pTs so that HT = 1
HT = np.sum(data[:,:,0],axis=-1)
data[:,:,0] = data[:,:,0]/HT[:,None]
data[:,:,-1] = data[:,:,-1]/HT[:,None]

# Center jet (optional)
data = center_jets_ptetaphiE(data)

# Inputs x to NN will be: pT, eta, cos(phi), sin(phi), log E
# Separated phi into cos and sin for continuity around full detector, so make things easier for NN.
# Also adding the log E is mainly because it seems like it should make things easier for NN, since there is an exponential spread in particle energies.
# Feel free to change these choices as desired. E.g. px, py might be equally as good as pt, sin, cos.
sig_input = np.zeros((len(data),50,5))
sig_input[:,:,:2] = data[:,:,:2]
sig_input[:,:,2] = np.cos(data[:,:,2])
sig_input[:,:,3] = np.sin(data[:,:,2])
sig_input[:,:,4] = np.log(data[:,:,3]+1e-8)


data_x = sig_input
# Event 'labels' y are [pT, eta, phi], which is used to calculate EMD to output which is also pT, eta, phi.
data_y = data[:,:,:3]


train_x = data_x[:500000]
train_y = data_y[:500000]
valid_x = data_x[500000:600000]
valid_y = data_y[500000:600000]

<ipython-input-2-c1d138f24178>:36: RuntimeWarning: invalid value encountered in true_divide
  y = 0.5*np.nan_to_num(np.log((E+pz)/(E-pz)))
<ipython-input-2-c1d138f24178>:57: RuntimeWarning: invalid value encountered in true_divide
  eta = np.nan_to_num(np.arcsinh(np.sinh(y)*np.sqrt(1+np.square(m/pt))))


In [47]:
output_dir = './data/'

experiment_name = 'W-test'
train_output_dir = create_dir(osp.join(output_dir, experiment_name))
vae, encoder, decoder = build_and_compile_annealing_vae(optimizer=keras.optimizers.Adam(lr=0.001,clipnorm=0.1),
                                    encoder_conv_layers = [2048,2048,1028,1024],
                                    dense_size = [1028,1028,1028,512],
                                    decoder = [4026,2048,1028,512,512],
                                    numItermaxinner = 40,   # EMD approximation params
                                    numIter=10,
                                    reg_init = 1.,
                                    reg_final = 0.01,
                                    stopThr=1e-3,
                                    num_inputs=5,           # Size of x (e.g. pT, eta, sin, cos, log E)
                                    num_particles_in=50)    # Num particles per event.

batch_size=100
save_period=2

reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=1e-4, cooldown=0, min_lr=0)
modelcheckpoint = keras.callbacks.ModelCheckpoint(train_output_dir + '/model_weights_{epoch:02d}.hdf5', save_freq = save_period*5000, save_weights_only=True)
reset_metrics_inst = reset_metrics()

callbacks=[tf.keras.callbacks.CSVLogger(train_output_dir + '/log.csv', separator=",", append=True),
            reduceLR,
            modelcheckpoint,
            reset_metrics_inst]


# Need to train on at least one example before model params can be loaded for annoying reasons.

history = vae.fit(x=train_x[:10], y=train_y[:10], batch_size=batch_size,
                epochs=1,verbose=1,#initial_epoch=int(vae.optimizer.iterations/numbatches),
                validation_data = (valid_x[:10],valid_y[:10]),
                callbacks = callbacks
              )

Model: "VAE"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 50, 5)]      0                                            
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 50, 2048)     12288       inputs[0][0]                     
__________________________________________________________________________________________________
re_lu_39 (ReLU)                 (None, 50, 2048)     0           conv1d_12[0][0]                  
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 50, 2048)     4196352     re_lu_39[0][0]                   
________________________________________________________________________________________________

In [48]:
init_epoch = 3957
beta = 1.6e-4
# vae.load_weights(train_output_dir + '/model_weights_end_' + str(init_epoch) + '_' + str(beta) + '.hdf5')
vae.load_weights(train_output_dir + '/model_weights_end_' + str(init_epoch) + '_' + "{:.1e}".format(beta) + '.hdf5')

In [49]:
# init_epoch = 544
steps_per_epoch = 1000
save_period = 10


reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5, verbose=1, mode='auto', min_delta=1e-4, cooldown=0, min_lr=1e-8)
earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0., patience=10, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)


for beta in np.logspace(-3,np.log10(4e-2),15):
    modelcheckpoint = keras.callbacks.ModelCheckpoint('/media/sda1/train-W' + '/model_weights_{epoch:02d}_' + "{:.1e}".format(beta) + '.hdf5', save_freq = save_period*steps_per_epoch, save_weights_only=True)
    callbacks=[tf.keras.callbacks.CSVLogger(train_output_dir + '/log.csv', separator=",", append=True),
            reduceLR,earlystop,
            modelcheckpoint,
            reset_metrics_inst]
    vae.beta.assign(beta)
    K.set_value(vae.optimizer.lr,3e-5)
    
    my_history = vae.fit(x=train_x, y=train_y, batch_size=batch_size,
                epochs=10000,verbose=1,
                validation_data = (valid_x[:200*batch_size],valid_y[:200*batch_size]),
                callbacks = callbacks,
                initial_epoch=init_epoch,
                steps_per_epoch = steps_per_epoch
              )
    init_epoch = my_history.epoch[-1]
    vae.save_weights(train_output_dir + '/model_weights_end_' + str(init_epoch) + '_' + "{:.1e}".format(beta) + '.hdf5')

Epoch 3958/10000
1000/1000 [==============================] - 157s 157ms/step - loss: 384.8230 - recon_loss: 3.3332e-04 - KL loss: 51.5036 - beta: 0.0010 - val_val_loss: 405.9765 - val_val_recon_loss: 3.6890e-04 - val_val_KL loss: 37.0715 - val_beta: 0.0010
Epoch 3959/10000
1000/1000 [==============================] - 145s 145ms/step - loss: 378.9243 - recon_loss: 3.4185e-04 - KL loss: 37.0766 - beta: 0.0010 - val_val_loss: 383.6831 - val_val_recon_loss: 3.4731e-04 - val_val_KL loss: 36.3773 - val_beta: 0.0010
Epoch 3960/10000
1000/1000 [==============================] - 147s 147ms/step - loss: 378.9336 - recon_loss: 3.4243e-04 - KL loss: 36.5063 - beta: 0.0010 - val_val_loss: 376.4502 - val_val_recon_loss: 3.4081e-04 - val_val_KL loss: 35.6357 - val_beta: 0.0010
Epoch 3961/10000
1000/1000 [==============================] - 147s 147ms/step - loss: 375.7469 - recon_loss: 3.4004e-04 - KL loss: 35.7102 - beta: 0.0010 - val_val_loss: 373.5909 - val_val_recon_loss: 3.3871e-04 - val_val_KL l

1000/1000 [==============================] - 157s 157ms/step - loss: 160.2492 - recon_loss: 3.8175e-04 - KL loss: 27.1907 - beta: 0.0017 - val_val_loss: 156.0661 - val_val_recon_loss: 3.7080e-04 - val_val_KL loss: 26.8232 - val_beta: 0.0017
Epoch 3985/10000
1000/1000 [==============================] - 156s 156ms/step - loss: 152.8389 - recon_loss: 3.6246e-04 - KL loss: 26.5019 - beta: 0.0017 - val_val_loss: 152.3174 - val_val_recon_loss: 3.6030e-04 - val_val_KL loss: 26.7338 - val_beta: 0.0017
Epoch 3986/10000
1000/1000 [==============================] - 157s 157ms/step - loss: 159.3390 - recon_loss: 3.8018e-04 - KL loss: 26.8278 - beta: 0.0017 - val_val_loss: 165.1037 - val_val_recon_loss: 3.9755e-04 - val_val_KL loss: 26.5348 - val_beta: 0.0017
Epoch 3987/10000
1000/1000 [==============================] - 157s 157ms/step - loss: 162.9314 - recon_loss: 3.9145e-04 - KL loss: 26.4917 - beta: 0.0017 - val_val_loss: 159.3339 - val_val_recon_loss: 3.8040e-04 - val_val_KL loss: 26.7459 - va

1000/1000 [==============================] - 156s 156ms/step - loss: 142.5600 - recon_loss: 3.3529e-04 - KL loss: 25.6951 - beta: 0.0017 - val_val_loss: 145.0309 - val_val_recon_loss: 3.4243e-04 - val_val_KL loss: 25.6755 - val_beta: 0.0017
Epoch 4015/10000
1000/1000 [==============================] - 157s 157ms/step - loss: 144.2151 - recon_loss: 3.3999e-04 - KL loss: 25.7108 - beta: 0.0017 - val_val_loss: 144.7515 - val_val_recon_loss: 3.4131e-04 - val_val_KL loss: 25.7885 - val_beta: 0.0017
Epoch 4016/10000
1000/1000 [==============================] - 157s 157ms/step - loss: 143.3266 - recon_loss: 3.3728e-04 - KL loss: 25.7665 - beta: 0.0017 - val_val_loss: 144.9068 - val_val_recon_loss: 3.4175e-04 - val_val_KL loss: 25.7877 - val_beta: 0.0017
Epoch 4017/10000
1000/1000 [==============================] - 156s 156ms/step - loss: 144.1237 - recon_loss: 3.3954e-04 - KL loss: 25.7758 - beta: 0.0017 - val_val_loss: 144.9988 - val_val_recon_loss: 3.4221e-04 - val_val_KL loss: 25.7221 - va

1000/1000 [==============================] - 156s 156ms/step - loss: 140.3836 - recon_loss: 3.2920e-04 - KL loss: 25.6400 - beta: 0.0017 - val_val_loss: 142.0058 - val_val_recon_loss: 3.3396e-04 - val_val_KL loss: 25.6042 - val_beta: 0.0017
Epoch 4045/10000
1000/1000 [==============================] - 156s 156ms/step - loss: 139.7889 - recon_loss: 3.2742e-04 - KL loss: 25.6663 - beta: 0.0017 - val_val_loss: 141.8732 - val_val_recon_loss: 3.3356e-04 - val_val_KL loss: 25.6093 - val_beta: 0.0017
Epoch 4046/10000
1000/1000 [==============================] - 157s 157ms/step - loss: 140.1793 - recon_loss: 3.2863e-04 - KL loss: 25.6332 - beta: 0.0017 - val_val_loss: 141.7715 - val_val_recon_loss: 3.3313e-04 - val_val_KL loss: 25.6579 - val_beta: 0.0017
Epoch 4047/10000
1000/1000 [==============================] - 157s 157ms/step - loss: 139.7910 - recon_loss: 3.2745e-04 - KL loss: 25.6589 - beta: 0.0017 - val_val_loss: 141.5583 - val_val_recon_loss: 3.3258e-04 - val_val_KL loss: 25.6369 - va

1000/1000 [==============================] - 157s 157ms/step - loss: 99.5954 - recon_loss: 3.7227e-04 - KL loss: 22.9897 - beta: 0.0022 - val_val_loss: 97.2200 - val_val_recon_loss: 3.6258e-04 - val_val_KL loss: 22.6094 - val_beta: 0.0022
Epoch 4073/10000
1000/1000 [==============================] - 157s 157ms/step - loss: 95.3490 - recon_loss: 3.5355e-04 - KL loss: 22.5966 - beta: 0.0022 - val_val_loss: 97.4821 - val_val_recon_loss: 3.6511e-04 - val_val_KL loss: 22.3503 - val_beta: 0.0022
Epoch 4074/10000
1000/1000 [==============================] - ETA: 0s - loss: 99.3654 - recon_loss: 3.7408e-04 - KL loss: 22.3873 - beta: 0.0022
Epoch 04074: ReduceLROnPlateau reducing learning rate to 9.486832740847579e-06.
1000/1000 [==============================] - 157s 157ms/step - loss: 99.3666 - recon_loss: 3.7409e-04 - KL loss: 22.3872 - beta: 0.0022 - val_val_loss: 99.9790 - val_val_recon_loss: 3.7489e-04 - val_val_KL loss: 22.8340 - val_beta: 0.0022
Epoch 4075/10000
1000/1000 [=============

...

1000/1000 [==============================] - 152s 152ms/step - loss: 9.4267 - recon_loss: 0.0014 - KL loss: 5.0713 - beta: 0.0181 - val_val_loss: 9.4257 - val_val_recon_loss: 0.0014 - val_val_KL loss: 5.0960 - val_beta: 0.0181
Epoch 4546/10000
1000/1000 [==============================] - 153s 153ms/step - loss: 9.3839 - recon_loss: 0.0014 - KL loss: 5.0547 - beta: 0.0181 - val_val_loss: 9.3850 - val_val_recon_loss: 0.0014 - val_val_KL loss: 4.9891 - val_beta: 0.0181
Epoch 4547/10000
1000/1000 [==============================] - 152s 152ms/step - loss: 9.4197 - recon_loss: 0.0014 - KL loss: 5.0573 - beta: 0.0181 - val_val_loss: 9.3957 - val_val_recon_loss: 0.0015 - val_val_KL loss: 4.9541 - val_beta: 0.0181
Epoch 4548/10000
1000/1000 [==============================] - 152s 152ms/step - loss: 9.4036 - recon_loss: 0.0014 - KL loss: 5.0409 - beta: 0.0181 - val_val_loss: 9.3503 - val_val_recon_loss: 0.0014 - val_val_KL loss: 5.0355 - val_beta: 0.0181
Epoch 4549/10000
1000/1000 [=============

Epoch 4578/10000
1000/1000 [==============================] - ETA: 0s - loss: 9.3233 - recon_loss: 0.0014 - KL loss: 5.0202 - beta: 0.0181
Epoch 04578: ReduceLROnPlateau reducing learning rate to 2.9999999502915785e-06.
1000/1000 [==============================] - 153s 153ms/step - loss: 9.3233 - recon_loss: 0.0014 - KL loss: 5.0202 - beta: 0.0181 - val_val_loss: 9.2985 - val_val_recon_loss: 0.0014 - val_val_KL loss: 5.0471 - val_beta: 0.0181
Epoch 4579/10000
1000/1000 [==============================] - 152s 152ms/step - loss: 9.2918 - recon_loss: 0.0014 - KL loss: 5.0109 - beta: 0.0181 - val_val_loss: 9.2602 - val_val_recon_loss: 0.0014 - val_val_KL loss: 5.0217 - val_beta: 0.0181
Epoch 4580/10000
1000/1000 [==============================] - 152s 152ms/step - loss: 9.2475 - recon_loss: 0.0014 - KL loss: 4.9987 - beta: 0.0181 - val_val_loss: 9.2674 - val_val_recon_loss: 0.0014 - val_val_KL loss: 4.9971 - val_beta: 0.0181
Epoch 4581/10000
1000/1000 [==============================] - 152

1000/1000 [==============================] - 149s 149ms/step - loss: 7.2717 - recon_loss: 0.0019 - KL loss: 3.8420 - beta: 0.0236 - val_val_loss: 7.2556 - val_val_recon_loss: 0.0019 - val_val_KL loss: 3.8552 - val_beta: 0.0236
Epoch 4608/10000
1000/1000 [==============================] - 150s 150ms/step - loss: 7.2492 - recon_loss: 0.0019 - KL loss: 3.8409 - beta: 0.0236 - val_val_loss: 7.2524 - val_val_recon_loss: 0.0019 - val_val_KL loss: 3.8447 - val_beta: 0.0236
Epoch 4609/10000
1000/1000 [==============================] - 150s 150ms/step - loss: 7.2847 - recon_loss: 0.0019 - KL loss: 3.8467 - beta: 0.0236 - val_val_loss: 7.2677 - val_val_recon_loss: 0.0019 - val_val_KL loss: 3.8524 - val_beta: 0.0236
Epoch 4610/10000
1000/1000 [==============================] - 149s 149ms/step - loss: 7.2457 - recon_loss: 0.0019 - KL loss: 3.8401 - beta: 0.0236 - val_val_loss: 7.2757 - val_val_recon_loss: 0.0019 - val_val_KL loss: 3.8314 - val_beta: 0.0236
Epoch 4611/10000
1000/1000 [=============

1000/1000 [==============================] - 147s 147ms/step - loss: 5.7511 - recon_loss: 0.0024 - KL loss: 3.1806 - beta: 0.0307 - val_val_loss: 5.7494 - val_val_recon_loss: 0.0024 - val_val_KL loss: 3.1851 - val_beta: 0.0307
Epoch 4637/10000
1000/1000 [==============================] - 147s 147ms/step - loss: 5.7516 - recon_loss: 0.0024 - KL loss: 3.1772 - beta: 0.0307 - val_val_loss: 5.7486 - val_val_recon_loss: 0.0024 - val_val_KL loss: 3.1877 - val_beta: 0.0307
Epoch 4638/10000
1000/1000 [==============================] - 146s 146ms/step - loss: 5.7601 - recon_loss: 0.0024 - KL loss: 3.1856 - beta: 0.0307 - val_val_loss: 5.7514 - val_val_recon_loss: 0.0025 - val_val_KL loss: 3.1430 - val_beta: 0.0307
Epoch 4639/10000
1000/1000 [==============================] - 147s 147ms/step - loss: 5.7429 - recon_loss: 0.0024 - KL loss: 3.1731 - beta: 0.0307 - val_val_loss: 5.7277 - val_val_recon_loss: 0.0024 - val_val_KL loss: 3.1775 - val_beta: 0.0307
Epoch 4640/10000
1000/1000 [=============

1000/1000 [==============================] - 147s 147ms/step - loss: 5.7314 - recon_loss: 0.0024 - KL loss: 3.1796 - beta: 0.0307 - val_val_loss: 5.7136 - val_val_recon_loss: 0.0024 - val_val_KL loss: 3.1819 - val_beta: 0.0307
Epoch 4667/10000
1000/1000 [==============================] - 147s 147ms/step - loss: 5.7331 - recon_loss: 0.0024 - KL loss: 3.1802 - beta: 0.0307 - val_val_loss: 5.7253 - val_val_recon_loss: 0.0024 - val_val_KL loss: 3.1806 - val_beta: 0.0307
Epoch 4668/10000
1000/1000 [==============================] - 146s 146ms/step - loss: 5.7226 - recon_loss: 0.0024 - KL loss: 3.1806 - beta: 0.0307 - val_val_loss: 5.7154 - val_val_recon_loss: 0.0024 - val_val_KL loss: 3.1812 - val_beta: 0.0307
Epoch 4669/10000
1000/1000 [==============================] - 146s 146ms/step - loss: 5.7161 - recon_loss: 0.0024 - KL loss: 3.1826 - beta: 0.0307 - val_val_loss: 5.7233 - val_val_recon_loss: 0.0024 - val_val_KL loss: 3.1807 - val_beta: 0.0307
Epoch 4670/10000
1000/1000 [=============

In [41]:
# init_epoch = 544
steps_per_epoch = 1000
save_period = 10


reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='val_val_loss', factor=np.sqrt(0.1), patience=5, verbose=1, mode='auto', min_delta=1e-4, cooldown=0, min_lr=1e-8)
earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_val_loss', min_delta=0., patience=10, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)


for beta in np.logspace(-1,-4,15)[2:]:
    modelcheckpoint = keras.callbacks.ModelCheckpoint('/media/sda1/train-W' + '/model_weights_{epoch:02d}_' + "{:.1e}".format(beta) + '.hdf5', save_freq = save_period*steps_per_epoch, save_weights_only=True)
    callbacks=[tf.keras.callbacks.CSVLogger(train_output_dir + '/log.csv', separator=",", append=True),
            reduceLR,earlystop,
            modelcheckpoint,
            reset_metrics_inst]
    vae.beta.assign(beta)
    K.set_value(vae.optimizer.lr,3e-5)
    
    my_history = vae.fit(x=train_x, y=train_y, batch_size=batch_size,
                epochs=10000,verbose=1,
                validation_data = (valid_x[:200*batch_size],valid_y[:200*batch_size]),
                callbacks = callbacks,
                initial_epoch=init_epoch,
                steps_per_epoch = steps_per_epoch
              )
    init_epoch = my_history.epoch[-1]
    vae.save_weights(train_output_dir + '/model_weights_end_' + str(init_epoch) + '_' + "{:.1e}".format(beta) + '.hdf5')

Epoch 3229/10000
1000/1000 [==============================] - 112s 112ms/step - loss: 6.9833 - recon_loss: 0.0088 - KL loss: 0.6476 - beta: 0.0373 - val_val_loss: 5.5028 - val_val_recon_loss: 0.0047 - val_val_KL loss: 2.0897 - val_beta: 0.0373
Epoch 3230/10000
1000/1000 [==============================] - 107s 107ms/step - loss: 5.3901 - recon_loss: 0.0043 - KL loss: 2.2608 - beta: 0.0373 - val_val_loss: 5.1368 - val_val_recon_loss: 0.0037 - val_val_KL loss: 2.4528 - val_beta: 0.0373
Epoch 3231/10000
1000/1000 [==============================] - 107s 107ms/step - loss: 5.0955 - recon_loss: 0.0036 - KL loss: 2.4881 - beta: 0.0373 - val_val_loss: 5.0536 - val_val_recon_loss: 0.0035 - val_val_KL loss: 2.5608 - val_beta: 0.0373
Epoch 3232/10000
1000/1000 [==============================] - 108s 108ms/step - loss: 5.0007 - recon_loss: 0.0034 - KL loss: 2.5638 - beta: 0.0373 - val_val_loss: 4.9813 - val_val_recon_loss: 0.0034 - val_val_KL loss: 2.4994 - val_beta: 0.0373
Epoch 3233/10000
1000/10

1000/1000 [==============================] - 115s 115ms/step - loss: 4.8171 - recon_loss: 0.0030 - KL loss: 2.6488 - beta: 0.0373 - val_val_loss: 4.8047 - val_val_recon_loss: 0.0029 - val_val_KL loss: 2.6823 - val_beta: 0.0373
Epoch 3263/10000
1000/1000 [==============================] - ETA: 0s - loss: 4.8231 - recon_loss: 0.0030 - KL loss: 2.6745 - beta: 0.0373
Epoch 03263: ReduceLROnPlateau reducing learning rate to 9.486832740847579e-06.
1000/1000 [==============================] - 115s 115ms/step - loss: 4.8231 - recon_loss: 0.0030 - KL loss: 2.6745 - beta: 0.0373 - val_val_loss: 4.8269 - val_val_recon_loss: 0.0030 - val_val_KL loss: 2.6437 - val_beta: 0.0373
Epoch 3264/10000
1000/1000 [==============================] - 116s 116ms/step - loss: 4.8028 - recon_loss: 0.0030 - KL loss: 2.6698 - beta: 0.0373 - val_val_loss: 4.7902 - val_val_recon_loss: 0.0029 - val_val_KL loss: 2.6900 - val_beta: 0.0373
Epoch 3265/10000
1000/1000 [==============================] - 116s 116ms/step - los

1000/1000 [==============================] - 117s 117ms/step - loss: 7.5183 - recon_loss: 0.0019 - KL loss: 3.8158 - beta: 0.0228 - val_val_loss: 7.4768 - val_val_recon_loss: 0.0019 - val_val_KL loss: 3.8621 - val_beta: 0.0228
Epoch 3293/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 7.5367 - recon_loss: 0.0019 - KL loss: 3.8202 - beta: 0.0228 - val_val_loss: 7.4715 - val_val_recon_loss: 0.0019 - val_val_KL loss: 3.7930 - val_beta: 0.0228
Epoch 3294/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 7.4983 - recon_loss: 0.0019 - KL loss: 3.8216 - beta: 0.0228 - val_val_loss: 7.5366 - val_val_recon_loss: 0.0019 - val_val_KL loss: 3.8061 - val_beta: 0.0228
Epoch 3295/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 7.4732 - recon_loss: 0.0019 - KL loss: 3.8207 - beta: 0.0228 - val_val_loss: 7.4388 - val_val_recon_loss: 0.0019 - val_val_KL loss: 3.8227 - val_beta: 0.0228
Epoch 3296/10000
1000/1000 [=============

1000/1000 [==============================] - 115s 115ms/step - loss: 7.4104 - recon_loss: 0.0019 - KL loss: 3.8273 - beta: 0.0228 - val_val_loss: 7.4115 - val_val_recon_loss: 0.0019 - val_val_KL loss: 3.8205 - val_beta: 0.0228
Epoch 3324/10000
1000/1000 [==============================] - 116s 116ms/step - loss: 7.4197 - recon_loss: 0.0019 - KL loss: 3.8308 - beta: 0.0228 - val_val_loss: 7.4003 - val_val_recon_loss: 0.0019 - val_val_KL loss: 3.8080 - val_beta: 0.0228
Epoch 3325/10000
1000/1000 [==============================] - ETA: 0s - loss: 7.4259 - recon_loss: 0.0019 - KL loss: 3.8290 - beta: 0.0228
Epoch 03325: ReduceLROnPlateau reducing learning rate to 2.9999998064878395e-07.
1000/1000 [==============================] - 116s 116ms/step - loss: 7.4259 - recon_loss: 0.0019 - KL loss: 3.8290 - beta: 0.0228 - val_val_loss: 7.4075 - val_val_recon_loss: 0.0019 - val_val_KL loss: 3.8258 - val_beta: 0.0228
Epoch 3325/10000
1000/1000 [==============================] - 116s 116ms/step - lo

1000/1000 [==============================] - ETA: 0s - loss: 12.7113 - recon_loss: 0.0015 - KL loss: 4.9308 - beta: 0.0139
Epoch 03353: ReduceLROnPlateau reducing learning rate to 2.9999999502915785e-06.
1000/1000 [==============================] - 116s 116ms/step - loss: 12.7113 - recon_loss: 0.0015 - KL loss: 4.9308 - beta: 0.0139 - val_val_loss: 12.6825 - val_val_recon_loss: 0.0015 - val_val_KL loss: 4.9008 - val_beta: 0.0139
Epoch 3354/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 12.6537 - recon_loss: 0.0015 - KL loss: 4.9187 - beta: 0.0139 - val_val_loss: 12.6300 - val_val_recon_loss: 0.0015 - val_val_KL loss: 4.9325 - val_beta: 0.0139
Epoch 3355/10000
1000/1000 [==============================] - 116s 116ms/step - loss: 12.5900 - recon_loss: 0.0015 - KL loss: 4.9256 - beta: 0.0139 - val_val_loss: 12.6536 - val_val_recon_loss: 0.0015 - val_val_KL loss: 4.9219 - val_beta: 0.0139
Epoch 3356/10000
1000/1000 [==============================] - 117s 117ms/st

...

Epoch 3916/10000
1000/1000 [==============================] - ETA: 0s - loss: 4508.4755 - recon_loss: 3.2037e-04 - KL loss: 56.9530 - beta: 2.6827e-04
Epoch 03916: ReduceLROnPlateau reducing learning rate to 2.9999999502915785e-06.
1000/1000 [==============================] - 117s 117ms/step - loss: 4508.4614 - recon_loss: 3.2037e-04 - KL loss: 56.9536 - beta: 2.6827e-04 - val_val_loss: 4636.9932 - val_val_recon_loss: 3.2955e-04 - val_val_KL loss: 57.9397 - val_beta: 2.6827e-04
Epoch 3917/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 4504.0267 - recon_loss: 3.1997e-04 - KL loss: 58.0904 - beta: 2.6827e-04 - val_val_loss: 4534.7139 - val_val_recon_loss: 3.2223e-04 - val_val_KL loss: 57.3796 - val_beta: 2.6827e-04
Epoch 3918/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 4471.4365 - recon_loss: 3.1766e-04 - KL loss: 57.6089 - beta: 2.6827e-04 - val_val_loss: 4536.2520 - val_val_recon_loss: 3.2228e-04 - val_val_KL loss: 58.1983 - va

1000/1000 [==============================] - 117s 117ms/step - loss: 11530.8748 - recon_loss: 3.0706e-04 - KL loss: 84.8613 - beta: 1.6379e-04 - val_val_loss: 11546.8740 - val_val_recon_loss: 3.0748e-04 - val_val_KL loss: 85.3773 - val_beta: 1.6379e-04
Epoch 3944/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 11568.6758 - recon_loss: 3.0808e-04 - KL loss: 84.6969 - beta: 1.6379e-04 - val_val_loss: 11655.3301 - val_val_recon_loss: 3.1038e-04 - val_val_KL loss: 85.4456 - val_beta: 1.6379e-04
Epoch 3945/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 11510.8236 - recon_loss: 3.0651e-04 - KL loss: 85.3547 - beta: 1.6379e-04 - val_val_loss: 11606.6748 - val_val_recon_loss: 3.0910e-04 - val_val_KL loss: 84.7797 - val_beta: 1.6379e-04
Epoch 3946/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 11353.9372 - recon_loss: 3.0232e-04 - KL loss: 84.5914 - beta: 1.6379e-04 - val_val_loss: 11530.3799 - val_val_recon_loss

Epoch 3970/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 30390.7032 - recon_loss: 3.0246e-04 - KL loss: 144.4160 - beta: 1.0000e-04 - val_val_loss: 31465.9316 - val_val_recon_loss: 3.1319e-04 - val_val_KL loss: 147.2839 - val_beta: 1.0000e-04
Epoch 3971/10000
1000/1000 [==============================] - ETA: 0s - loss: 30605.4073 - recon_loss: 3.0460e-04 - KL loss: 145.3790 - beta: 1.0000e-04
Epoch 03971: ReduceLROnPlateau reducing learning rate to 2.9999999502915785e-06.
1000/1000 [==============================] - 117s 117ms/step - loss: 30605.1529 - recon_loss: 3.0460e-04 - KL loss: 145.3785 - beta: 1.0000e-04 - val_val_loss: 30703.7969 - val_val_recon_loss: 3.0558e-04 - val_val_KL loss: 145.9330 - val_beta: 1.0000e-04
Epoch 3972/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 30368.4104 - recon_loss: 3.0222e-04 - KL loss: 145.9436 - beta: 1.0000e-04 - val_val_loss: 30479.6797 - val_val_recon_loss: 3.0333e-04 - val_val_KL loss:

1000/1000 [==============================] - 117s 117ms/step - loss: 29507.2187 - recon_loss: 2.9360e-04 - KL loss: 147.2308 - beta: 1.0000e-04 - val_val_loss: 29870.1191 - val_val_recon_loss: 2.9723e-04 - val_val_KL loss: 147.3313 - val_beta: 1.0000e-04
Epoch 4000/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 29181.1937 - recon_loss: 2.9034e-04 - KL loss: 146.8915 - beta: 1.0000e-04 - val_val_loss: 29768.0801 - val_val_recon_loss: 2.9621e-04 - val_val_KL loss: 147.2242 - val_beta: 1.0000e-04
Epoch 4001/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 29209.7714 - recon_loss: 2.9064e-04 - KL loss: 145.6180 - beta: 1.0000e-04 - val_val_loss: 29666.2266 - val_val_recon_loss: 2.9520e-04 - val_val_KL loss: 146.5338 - val_beta: 1.0000e-04
Epoch 4002/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 29174.4734 - recon_loss: 2.9028e-04 - KL loss: 146.6573 - beta: 1.0000e-04 - val_val_loss: 29675.6953 - val_val_rec

Epoch 4028/10000
1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 1.0000e-04 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 1.0000e-04
Epoch 4029/10000
1000/1000 [==============================] - ETA: 0s - loss: nan - recon_loss: nan - KL loss: nan - beta: 1.0000e-04
Epoch 04029: ReduceLROnPlateau reducing learning rate to 2.9999998963651765e-08.
1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 1.0000e-04 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 1.0000e-04


In [25]:
# init_epoch = 544
steps_per_epoch = 1000
save_period = 10


reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='val_val_loss', factor=np.sqrt(0.1), patience=5, verbose=1, mode='auto', min_delta=1e-4, cooldown=0, min_lr=1e-8)
earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_val_loss', min_delta=0., patience=10, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)


for beta in np.concatenate((np.logspace(-4,np.log10(0.7),15),
               np.logspace(np.log10(0.7),-5,20)[1:],
                np.logspace(-5,np.log10(0.7),20)[1:])):
    modelcheckpoint = keras.callbacks.ModelCheckpoint('/media/sda1/train-W' + '/model_weights_{epoch:02d}_' + "{:.1e}".format(beta) + '.hdf5', save_freq = save_period*steps_per_epoch, save_weights_only=True)
    callbacks=[tf.keras.callbacks.CSVLogger(train_output_dir + '/log.csv', separator=",", append=True),
            reduceLR,earlystop,
            modelcheckpoint,
            reset_metrics_inst]
    vae.beta.assign(beta)
    K.set_value(vae.optimizer.lr,3e-5)
    
    my_history = vae.fit(x=train_x, y=train_y, batch_size=batch_size,
                epochs=10000,verbose=1,
                validation_data = (valid_x[:200*batch_size],valid_y[:200*batch_size]),
                callbacks = callbacks,
                initial_epoch=init_epoch,
                steps_per_epoch = steps_per_epoch
              )
    init_epoch = my_history.epoch[-1]
    vae.save_weights(train_output_dir + '/model_weights_end_' + str(init_epoch) + '_' + "{:.1e}".format(beta) + '.hdf5')

Epoch 756/10000
1000/1000 [==============================] - 116s 116ms/step - loss: 31734.9253 - recon_loss: 3.1504e-04 - KL loss: 231.1730 - beta: 1.0000e-04 - val_val_loss: 32552.8555 - val_val_recon_loss: 3.2328e-04 - val_val_KL loss: 224.8359 - val_beta: 1.0000e-04
Epoch 757/10000
1000/1000 [==============================] - 109s 109ms/step - loss: 31422.9057 - recon_loss: 3.1198e-04 - KL loss: 225.1942 - beta: 1.0000e-04 - val_val_loss: 32635.0020 - val_val_recon_loss: 3.2409e-04 - val_val_KL loss: 225.6331 - val_beta: 1.0000e-04
Epoch 758/10000
1000/1000 [==============================] - 111s 111ms/step - loss: 32146.5800 - recon_loss: 3.1916e-04 - KL loss: 230.2496 - beta: 1.0000e-04 - val_val_loss: 34566.9961 - val_val_recon_loss: 3.4329e-04 - val_val_KL loss: 237.7053 - val_beta: 1.0000e-04
Epoch 759/10000
1000/1000 [==============================] - 114s 114ms/step - loss: 34457.7641 - recon_loss: 3.4222e-04 - KL loss: 235.7530 - beta: 1.0000e-04 - val_val_loss: 33953.9844 

Epoch 783/10000
1000/1000 [==============================] - 116s 116ms/step - loss: 9695.9676 - recon_loss: 3.3708e-04 - KL loss: 180.3865 - beta: 1.8821e-04 - val_val_loss: 9603.8359 - val_val_recon_loss: 3.3405e-04 - val_val_KL loss: 173.7154 - val_beta: 1.8821e-04
Epoch 784/10000
1000/1000 [==============================] - 115s 115ms/step - loss: 9490.7684 - recon_loss: 3.3011e-04 - KL loss: 172.1215 - beta: 1.8821e-04 - val_val_loss: 9527.5752 - val_val_recon_loss: 3.3166e-04 - val_val_KL loss: 165.1717 - val_beta: 1.8821e-04
Epoch 785/10000
1000/1000 [==============================] - 111s 111ms/step - loss: 9402.9572 - recon_loss: 3.2723e-04 - KL loss: 165.3779 - beta: 1.8821e-04 - val_val_loss: 9534.2842 - val_val_recon_loss: 3.3178e-04 - val_val_KL loss: 168.4199 - val_beta: 1.8821e-04
Epoch 786/10000
1000/1000 [==============================] - 109s 109ms/step - loss: 9340.7008 - recon_loss: 3.2503e-04 - KL loss: 165.3658 - beta: 1.8821e-04 - val_val_loss: 9195.4521 - val_va

1000/1000 [==============================] - 118s 118ms/step - loss: 8960.3873 - recon_loss: 3.1175e-04 - KL loss: 159.8263 - beta: 1.8821e-04 - val_val_loss: 9002.4404 - val_val_recon_loss: 3.1317e-04 - val_val_KL loss: 161.8598 - val_beta: 1.8821e-04
Epoch 813/10000
1000/1000 [==============================] - 118s 118ms/step - loss: 8987.7007 - recon_loss: 3.1270e-04 - KL loss: 160.2957 - beta: 1.8821e-04 - val_val_loss: 9065.2637 - val_val_recon_loss: 3.1541e-04 - val_val_KL loss: 161.4455 - val_beta: 1.8821e-04
Epoch 814/10000
1000/1000 [==============================] - ETA: 0s - loss: 9032.8043 - recon_loss: 3.1419e-04 - KL loss: 163.3644 - beta: 1.8821e-04
Epoch 00814: ReduceLROnPlateau reducing learning rate to 2.9999999502915785e-06.
1000/1000 [==============================] - 118s 118ms/step - loss: 9032.8269 - recon_loss: 3.1419e-04 - KL loss: 163.3644 - beta: 1.8821e-04 - val_val_loss: 9055.6562 - val_val_recon_loss: 3.1505e-04 - val_val_KL loss: 161.8716 - val_beta: 1.88

1000/1000 [==============================] - 117s 117ms/step - loss: 2377.9861 - recon_loss: 2.8740e-04 - KL loss: 87.7404 - beta: 3.5424e-04 - val_val_loss: 2414.1265 - val_val_recon_loss: 2.9189e-04 - val_val_KL loss: 88.0672 - val_beta: 3.5424e-04
Epoch 840/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 2384.8154 - recon_loss: 2.8822e-04 - KL loss: 87.9728 - beta: 3.5424e-04 - val_val_loss: 2418.5300 - val_val_recon_loss: 2.9241e-04 - val_val_KL loss: 88.3051 - val_beta: 3.5424e-04
Epoch 841/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 2394.0915 - recon_loss: 2.8941e-04 - KL loss: 87.8014 - beta: 3.5424e-04 - val_val_loss: 2402.1157 - val_val_recon_loss: 2.9049e-04 - val_val_KL loss: 87.2023 - val_beta: 3.5424e-04
Epoch 842/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 2357.0481 - recon_loss: 2.8485e-04 - KL loss: 87.1211 - beta: 3.5424e-04 - val_val_loss: 2399.1523 - val_val_recon_loss: 2.9011e-0

Epoch 870/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 2332.3216 - recon_loss: 2.8177e-04 - KL loss: 86.9308 - beta: 3.5424e-04 - val_val_loss: 2361.6855 - val_val_recon_loss: 2.8556e-04 - val_val_KL loss: 86.0360 - val_beta: 3.5424e-04
Epoch 871/10000
1000/1000 [==============================] - 117s 117ms/step - loss: 2319.6263 - recon_loss: 2.8028e-04 - KL loss: 86.0771 - beta: 3.5424e-04 - val_val_loss: 2358.6091 - val_val_recon_loss: 2.8514e-04 - val_val_KL loss: 86.3215 - val_beta: 3.5424e-04
Epoch 872/10000
1000/1000 [==============================] - 118s 118ms/step - loss: 2324.0502 - recon_loss: 2.8078e-04 - KL loss: 86.5661 - beta: 3.5424e-04 - val_val_loss: 2358.6531 - val_val_recon_loss: 2.8506e-04 - val_val_KL loss: 86.9938 - val_beta: 3.5424e-04
Epoch 873/10000
1000/1000 [==============================] - ETA: 0s - loss: 2331.8936 - recon_loss: 2.8173e-04 - KL loss: 86.7872 - beta: 3.5424e-04
Epoch 00873: ReduceLROnPlateau reducing learning 

...

1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 1.8838e-04 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 1.8838e-04
Epoch 2590/10000
1000/1000 [==============================] - 128s 128ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 1.8838e-04 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 1.8838e-04
Epoch 2591/10000
1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 1.8838e-04 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 1.8838e-04
Epoch 2592/10000
1000/1000 [==============================] - ETA: 0s - loss: nan - recon_loss: nan - KL loss: nan - beta: 1.8838e-04
Epoch 02592: ReduceLROnPlateau reducing learning rate to 9.486832740847579e-06.
1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss:

1000/1000 [==============================] - 128s 128ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0011 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 0.0011
Epoch 2617/10000
1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0011 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 0.0011
Epoch 2618/10000
1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0011 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 0.0011
Epoch 2619/10000
1000/1000 [==============================] - ETA: 0s - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0011
Epoch 02619: ReduceLROnPlateau reducing learning rate to 9.486832740847579e-06.
1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0011 - val_va

1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0064 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 0.0064
Epoch 2645/10000
1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0064 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 0.0064
Epoch 2646/10000
1000/1000 [==============================] - ETA: 0s - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0064
Epoch 02646: ReduceLROnPlateau reducing learning rate to 9.486832740847579e-06.
1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0064 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 0.0064
Epoch 2647/10000
1000/1000 [==============================] - 128s 128ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0064 - val_va

1000/1000 [==============================] - 128s 128ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0372 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 0.0372
Epoch 2673/10000
1000/1000 [==============================] - ETA: 0s - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0372
Epoch 02673: ReduceLROnPlateau reducing learning rate to 9.486832740847579e-06.
1000/1000 [==============================] - 128s 128ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0372 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 0.0372
Epoch 2674/10000
1000/1000 [==============================] - 128s 128ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0372 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 0.0372
Epoch 2675/10000
1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.0372 - val_va

1000/1000 [==============================] - ETA: 0s - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.2163
Epoch 02700: ReduceLROnPlateau reducing learning rate to 9.486832740847579e-06.
1000/1000 [==============================] - 128s 128ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.2163 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 0.2163
Epoch 2701/10000
1000/1000 [==============================] - 128s 128ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.2163 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 0.2163
Epoch 2702/10000
1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.2163 - val_val_loss: nan - val_val_recon_loss: nan - val_val_KL loss: nan - val_beta: 0.2163
Epoch 2703/10000
1000/1000 [==============================] - 129s 129ms/step - loss: nan - recon_loss: nan - KL loss: nan - beta: 0.2163 - val_va

In [27]:
my_history

NameError: name 'my_history' is not defined

In [21]:
data[344577]

array([[ 5.94342112e-01, -8.01087950e-03, -9.59239693e-02,
         5.94732045e-01],
       [ 3.20266890e-01,  1.36259927e-02,  1.71835828e-01,
         3.20508258e-01],
       [ 2.70634347e-02, -5.70251954e-02,  2.03458772e-01,
         2.71074501e-02],
       [ 1.67915073e-02, -5.83480486e-02,  1.27207984e-01,
         1.68200987e-02],
       [ 1.65186960e-02,  5.90673387e-02, -1.34921235e-01,
         1.65475208e-02],
       [ 4.71925992e-03,  9.60208657e-01,  2.47662626e-01,
         7.06722029e-03],
       [ 4.24085623e-03, -6.37045126e-02, -1.61667501e-01,
         4.24946440e-03],
       [ 3.17943365e-03, -3.47312263e-01, -8.84293165e-01,
         3.37312991e-03],
       [ 3.15840949e-03,  1.84969735e-02, -2.06772569e-01,
         3.15894981e-03],
       [ 2.70254265e-03, -8.54734571e-01,  1.71867674e-01,
         3.75132707e-03],
       [ 1.88874218e-03,  7.52684646e-01, -2.48769901e-01,
         2.44950139e-03],
       [ 1.53140233e-03, -5.97810065e-01, -2.43046386e-01,
      

In [13]:
train_x[344599:344699]

array([[[ 4.75817262e-01,  2.66602718e-02,  9.90336365e-01,
         -1.38686278e-01, -7.42237833e-01],
        [ 3.78478884e-01, -1.23382600e-02,  9.84472615e-01,
          1.75538232e-01, -9.71309188e-01],
        [ 3.88847388e-02, -5.61368658e-02,  9.98044277e-01,
          6.25109604e-02, -3.24557832e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
          0.00000000e+00, -1.84206807e+01],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
          0.00000000e+00, -1.84206807e+01],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
          0.00000000e+00, -1.84206807e+01]],

       [[ 9.21532888e-01, -3.50221957e-02,  9.99996328e-01,
         -2.70998044e-03, -8.09998991e-02],
        [ 1.66717116e-02,  1.09351969e-01,  9.99491337e-01,
          3.18914776e-02, -4.08241210e+00],
        [ 1.05050968e-02, -3.16219896e-02,  9.90696950e-01,
          1.36086569e-01, -4.54110070e+00],
        ...,
        [ 0.00000000e+00,  0.0000000

In [ ]:
train_output_dir + '/model_weights_{epoch:02d}_' + str(beta) + '.hdf5'